In [119]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, to_json
from pyspark.sql.types import StringType, StructType, StructField,IntegerType



In [120]:

# Define the schema for the incoming Kafka messages
schema = StructType([
    StructField("id", StringType(), True),
    StructField("message", StringType(), True)
])


In [121]:

# Initialize SparkSession with Kafka package
spark = SparkSession.builder \
    .appName("KafkaSparkStreaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1") \
    .getOrCreate()

# Set log level to WARN to reduce console output
spark.sparkContext.setLogLevel("WARN")


In [122]:

# Define Kafka parameters
kafka_bootstrap_servers = "172.31.26.78:9092"
kafka_topic = "spark-cluster"


In [124]:

# Read data from Kafka
kafka_df = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets","earliest") \
    .load()


In [125]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [126]:
df = kafka_df.selectExpr("CAST(value AS STRING)") 

In [127]:
schema = StructType() \
    .add("author", StringType()) \
    .add("comments", StringType()) \
    .add("created", StringType()) \
    .add("id", StringType()) \
    .add("num_comments", StringType()) \
    .add("score", StringType()) \
    .add("title", StringType()) 
    

In [128]:
df2 = df.select(from_json(col("value"),schema=schema))
df2.printSchema()

root
 |-- from_json(value): struct (nullable = true)
 |    |-- author: string (nullable = true)
 |    |-- comments: string (nullable = true)
 |    |-- created: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- num_comments: string (nullable = true)
 |    |-- score: string (nullable = true)
 |    |-- title: string (nullable = true)



In [129]:
df2.show()

Py4JJavaError: An error occurred while calling o438.showString.
: java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.TimeoutException: Call(callName=describeTopics, deadlineMs=1720389944495, tries=1, nextAllowedTryMs=1720389944598) timed out at 1720389944498 after 1 attempt(s)
	at org.apache.kafka.common.internals.KafkaFutureImpl.wrapAndThrow(KafkaFutureImpl.java:45)
	at org.apache.kafka.common.internals.KafkaFutureImpl.access$000(KafkaFutureImpl.java:32)
	at org.apache.kafka.common.internals.KafkaFutureImpl$SingleWaiter.await(KafkaFutureImpl.java:89)
	at org.apache.kafka.common.internals.KafkaFutureImpl.get(KafkaFutureImpl.java:260)
	at org.apache.spark.sql.kafka010.ConsumerStrategy.retrieveAllPartitions(ConsumerStrategy.scala:66)
	at org.apache.spark.sql.kafka010.ConsumerStrategy.retrieveAllPartitions$(ConsumerStrategy.scala:65)
	at org.apache.spark.sql.kafka010.SubscribeStrategy.retrieveAllPartitions(ConsumerStrategy.scala:102)
	at org.apache.spark.sql.k